## Question 1
Load the 49 industries Value weighted returns and cap weights, and use the period 2013-2018 both included. For the period, use the starting cap weights of the period. Limit yourself to the following 5 industry sectors: 'Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food'.

 You will need to compute the correlation matrix as well as the volatilities. (Hint: Remember to annualize the volatilities by multiplying the volatility you get from the monthly data by the sqrt iof 12)
 
Using the same value of delta used in the He-Litterman paper of 2.5 and using the same sigma prior methodology used in the notebook and in the paper, compute the implied returns vector.

Which industry sector has the highest capweight?  

Rtail

## Question 2
Use the same data as the previous question, which industry sector has the highest implied return?  

Rtail

## Question 3
Question 3
Use the same data and assumptions as the previous question. Which industry sector has the lowest implied return?   

Hlth

In [58]:
import edhec_risk_kit_206 as erk

import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

inds = ['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']
ind_rets = erk.get_ind_returns(weighting="vw", n_inds=49)["2013":"2018"][inds]
ind_mcap = erk.get_ind_market_caps(49, weights=True)["2013":"2018"][inds]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
ind_rets

,Hlth,Fin,Whlsl,Rtail,Food
2013-01,0.0966,0.1300,0.0646,0.0541,0.0754
2013-02,0.0078,0.0177,0.0207,0.0061,0.0483
2013-03,0.0591,0.0360,0.0325,0.0477,0.0798
2013-04,-0.0034,-0.0002,0.0017,0.0320,0.0174
2013-05,0.0454,0.0842,0.0385,0.0239,-0.0201
...,...,...,...,...,...
2018-08,0.0585,0.0004,0.0366,0.0911,0.0114
2018-09,-0.0045,-0.0340,-0.0148,0.0061,-0.0220
2018-10,-0.0659,-0.0455,-0.0788,-0.1021,0.0095
2018-11,0.0356,0.0006,0.0318,0.0159,0.0029


In [68]:
rho = ind_rets.loc[:,inds].corr()
rho

,Hlth,Fin,Whlsl,Rtail,Food
Hlth,1.000000,0.524886,0.650900,0.574227,0.371698
Fin,0.524886,1.000000,0.777653,0.595332,0.343016
Whlsl,0.650900,0.777653,1.000000,0.743754,0.541244
Rtail,0.574227,0.595332,0.743754,1.000000,0.575063
Food,0.371698,0.343016,0.541244,0.575063,1.000000


In [65]:
annualized_vol = ind_rets.std()*(12**0.5)
annualized_vol

Hlth     0.162801
Fin      0.167753
Whlsl    0.135644
Rtail    0.141238
Food     0.125401
dtype: float64

In [66]:
vols = pd.Series([0.162801,0.167753,0.135644,0.141238,0.125401],index=inds) 
vols

Hlth     0.162801
Fin      0.167753
Whlsl    0.135644
Rtail    0.141238
Food     0.125401
dtype: float64

In [85]:
# Compute the Covariance Matrix
sigma_prior = vols.dot(vols.T) * rho
sigma_prior

,Hlth,Fin,Whlsl,Rtail,Food
Hlth,0.108718,0.057065,0.070765,0.062429,0.040410
Fin,0.057065,0.108718,0.084545,0.064723,0.037292
Whlsl,0.070765,0.084545,0.108718,0.080860,0.058843
Rtail,0.062429,0.064723,0.080860,0.108718,0.062520
Food,0.040410,0.037292,0.058843,0.062520,0.108718


In [71]:
ind_mcap.sum()

Hlth     0.396624
Fin      1.751218
Whlsl    0.937026
Rtail    5.057213
Food     1.241093
dtype: float64

In [42]:
sum({0.396624,1.751218,0.937026,5.057213,1.241093})

9.383174

In [72]:
w_eq = pd.DataFrame([0.396624/9.383174,1.751218/9.383174,0.937026/9.383174,5.057213/9.383174,1.241093/9.383174], index=inds, columns=["CapWeight"])
w_eq

,CapWeight
Hlth,0.042270
Fin,0.186634
Whlsl,0.099862
Rtail,0.538966
Food,0.132268


In [73]:
def implied_returns(delta, sigma, w):
    """
Obtain the implied expected returns by reverse engineering the weights
Inputs:
delta: Risk Aversion Coefficient (scalar)
sigma: Variance-Covariance Matrix (N x N) as DataFrame
    w: Portfolio weights (N x 1) as Series
Returns an N x 1 vector of Returns as Series
    """
    ir = delta*sigma.dot(w).squeeze() # to get a series from a 1-column dataframe
    ir.name = 'Implied Returns'
    return ir

In [74]:
 # Compute Pi and compare:
pi = implied_returns(delta=2.5, sigma=sigma_prior, w=w_eq)
(pi*100).round(1)

Hlth     15.3
Fin      17.7
Whlsl    20.2
Rtail    22.4
Food     15.7
Name: Implied Returns, dtype: float64

## Question 4
Impose the subjective relative view that Hlth will outperform Rtail and Whlsl by 3%  (Hint: Use the same logic as View 1 in the He-Litterman paper)
What is the entry you will use for the Pick Matrix P for Whlsl. (Hint: Remember to use the correct sign)

-0.16

## Question 5
Impose the subjective relative view that Hlth will outperform Rtail and Whlsl by 3%  (Hint: Use the same logic as View 1 in the He-Litterman paper)
What is the entry you will use for the Pick Matrix P for Rtail. (Hint: Remember to use the correct sign)

-0.84
## Question 6
Impose the subjective relative view that Hlth will outperform Rtail and Whlsl by 3%  (Hint: Use the same logic as View 1 in the He-Litterman paper)
Once you impose this view (use delta = 2.5 and tau = 0.05 as in the paper), which sector has the lowest implied return?

Food

## Question 7
Impose the subjective relative view that Hlth will outperform Rtail and Whlsl by 3%  (Hint: Use the same logic as View 1 in the He-Litterman paper)
Which sector now has the highest weight in the MSR portfolio using the Black-Litterman model?

not Hlth not Fin???

Rtail

## Question 8
Impose the subjective relative view that Hlth will outperform Rtail and Whlsl by 3%  (Hint: Use the same logic as View 1 in the He-Litterman paper)
Which sector now has the lowest weight in the MSR portfolio using the Black-Litterman model?

'Whlsl'

In [75]:
# Hlth will outperform Rtail and Whlsl by 3%
q = pd.Series([.03]) # just one view
# start with a single view, all zeros and overwrite the specific view
p = pd.DataFrame([0.]*len(inds), index=inds).T
# find the relative market caps of Rtail and Whlsl to split the relative outperformance of Hlth ...
w_Rtail =  w_eq.loc["Rtail"]/(w_eq.loc["Rtail"]+w_eq.loc["Whlsl"])
w_Whlsl =  w_eq.loc["Whlsl"]/(w_eq.loc["Rtail"]+w_eq.loc["Whlsl"])
p.iloc[0]['Hlth'] = 1.
p.iloc[0]['Rtail'] = -w_Rtail
p.iloc[0]['Whlsl'] = -w_Whlsl
(p*100).round(1)

,Hlth,Fin,Whlsl,Rtail,Food
0,100.0,0.0,-15.6,-84.4,0.0


In [77]:
# Assumes that Omega is proportional to the variance of the prior
def proportional_prior(sigma, tau, p):
    """
    Returns the He-Litterman simplified Omega
    Inputs:
    sigma: N x N Covariance Matrix as DataFrame
    tau: a scalar
    p: a K x N DataFrame linking Q and Assets
    returns a P x P DataFrame, a Matrix representing Prior Uncertainties
    """
    helit_omega = p.dot(tau * sigma).dot(p.T)
    # Make a diag matrix from the diag elements of Omega
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index = p.index, columns = p.index)

In [78]:
from numpy.linalg import inv

def bl(w_prior, sigma_prior, p, q,
      omega=None,
      delta=2.5, tau=.02):
    """
# Computes the posterior expected returns based on 
# the original black litterman reference model
#
# W.prior must be an N x 1 vector of weights, a Series
# Sigma.prior is an N x N covariance matrix, a DataFrame
# P must be a K x N matrix linking Q and the Assets, a DataFrame
# Q must be an K x 1 vector of views, a Series
# Omega must be a K x K matrix a DataFrame, or None
# if Omega is None, we assume it is
#    proportional to variance of the prior
# delta and tau are scalars
    """
    if omega is None:
        omega = proportional_prior(sigma_prior, tau, p)
    # Force w.prior and Q to be column vectors
    # How many assets do we have?
    N = w_prior.shape[0]
    # And how many views?
    K = q.shape[0]
    # First, reverse-engineer the weights to get pi
    pi = implied_returns(delta, sigma_prior,  w_prior)
    # Adjust (scale) Sigma by the uncertainty scaling factor
    sigma_prior_scaled = tau * sigma_prior  
    # posterior estimate of the mean, use the "Master Formula"
    # we use the versions that do not require
    # Omega to be inverted (see previous section)
    # this is easier to read if we use '@' for matrixmult instead of .dot()
    #     mu_bl = pi + sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ (q - p @ pi)
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    # posterior estimate of uncertainty of mu.bl
#     sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ p @ sigma_prior_scaled
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl, sigma_bl)


In [79]:
# for convenience and readability, define the inverse of a dataframe
def inverse(d):
    """
    Invert the dataframe by inverting the underlying matrix
    """
    return pd.DataFrame(inv(d.values), index=d.columns, columns=d.index)

def w_msr(sigma, mu, scale=True):
    """
    Optimal (Tangent/Max Sharpe Ratio) Portfolio weights
    by using the Markowitz Optimization Procedure
    Mu is the vector of Excess expected Returns
    Sigma must be an N x N matrix as a DataFrame and Mu a column vector as a Series
    This implements page 188 Equation 5.2.28 of
    "The econometrics of financial markets" Campbell, Lo and Mackinlay.
    """
    w = inverse(sigma).dot(mu)
    if scale:
        w = w/sum(w) # fix: this assumes all w is +ve
    return w

In [80]:
delta = 2.5
tau = 0.05
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = bl(w_eq, sigma_prior, p, q, tau = tau)
(bl_mu*100).round(1)

Hlth     18.0
Fin      17.1
Whlsl    19.4
Rtail    19.9
Food     14.4
dtype: float64

In [81]:
def w_star(delta,sigma,mu):
    return(inverse(sigma).dot(mu))/delta

wstar = w_star(delta=2.5, sigma=bl_sigma, mu=bl_mu)
# display w*
(wstar*100).round(1)

Hlth     26.3
Fin      17.8
Whlsl     6.0
Rtail    32.5
Food     12.6
dtype: float64

## Question 9
Now, let’s assume you change the relative view. You still think that it Hlth will outperform Rtail and Whlsl but you think that the outperformance will be 5% not the 3% you originally anticipated.
Which of the arrays will you need to update?    

Q but not P

## Question 10
Now, let’s assume you change the relative view. You still think that it Hlth will outperform Rtail and Whlsl but you think that the outperformance will be 5% not the 3% you originally anticipated.
Under this new view which sector has the highest expected return?

Rtail

## Question 11
Now, let’s assume you change the relative view. You still think that it Hlth will outperform Rtail and Whlsl but you think that the outperformance will be 5% not the 3% you originally anticipated.
Under this new view which sector does the Black-Litterman model assign the highest weight?

Hlth

In [82]:
# Hlth will outperform Rtail and Whlsl by 5%
q = pd.Series([.05]) # just one view
# start with a single view, all zeros and overwrite the specific view
p = pd.DataFrame([0.]*len(inds), index=inds).T
# find the relative market caps of Rtail and Whlsl to split the relative outperformance of Hlth ...
w_Rtail =  w_eq.loc["Rtail"]/(w_eq.loc["Rtail"]+w_eq.loc["Whlsl"])
w_Whlsl =  w_eq.loc["Whlsl"]/(w_eq.loc["Rtail"]+w_eq.loc["Whlsl"])
p.iloc[0]['Hlth'] = 1.
p.iloc[0]['Rtail'] = -w_Rtail
p.iloc[0]['Whlsl'] = -w_Whlsl
(p*100).round(1)

,Hlth,Fin,Whlsl,Rtail,Food
0,100.0,0.0,-15.6,-84.4,0.0


In [83]:
delta = 2.5
tau = 0.05
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = bl(w_eq, sigma_prior, p, q, tau = tau)
(bl_mu*100).round(1)

Hlth     18.5
Fin      17.0
Whlsl    19.2
Rtail    19.4
Food     14.1
dtype: float64

In [84]:
def w_star(delta,sigma,mu):
    return(inverse(sigma).dot(mu))/delta

wstar = w_star(delta=2.5, sigma=bl_sigma, mu=bl_mu)
# display w*
(wstar*100).round(1)

Hlth     31.0
Fin      17.8
Whlsl     5.3
Rtail    28.6
Food     12.6
dtype: float64